# Scaling historical climate normals to 2100 using GCM projections

In [162]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
from sklearn.svm import SVR
from natsort import natsorted
import geopandas as gpd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
import HydroErr as he
import os
import statsmodels.api as sm

In [2]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 140

## Import data

In [664]:
# Average PRISM and Naselle gauge precipitation
gauge = pd.read_csv(config.daily_ppt.parents[0] / 'GHCND_USC00455774_1929_2020.csv', parse_dates=True, index_col=5)
gauge['SNOW'].fillna(0, inplace=True)
gauge['SNOW_SWE'] = gauge['SNOW'] / 13
gauge['PRCP_TOT'] = gauge['PRCP'] + gauge['SNOW_SWE']
gauge_precip = gauge[['PRCP_TOT']]

prism_precip = pd.read_csv(str(config.daily_ppt.parents[0] / 'prism_ppt_1981_2020_daily.csv'), parse_dates=True, index_col=0)

# Expand precip record to full date range in case some days are missing
hist_start = pd.to_datetime('01-01-1981')
hist_end = pd.to_datetime('12-31-2020')
rng = pd.date_range(hist_start, hist_end)
date_df = pd.DataFrame(index=rng)
gauge_precip_velma = date_df.merge(gauge_precip, left_index=True, right_index=True, how='left')
prism_precip_velma = date_df.merge(prism_precip, left_index=True, right_index=True, how='left')

prism_precip_mean = pd.concat([gauge_precip_velma, prism_precip_velma], axis=1)
prism_precip_mean['avg_precip'] = prism_precip_mean.mean(axis=1)

# Air temperature
temp_path = config.daily_temp_mean.parents[0] / 'prism_temp_1981_2020_daily.csv'
temp = pd.read_csv(temp_path, parse_dates=True, index_col=0)

C:\Users\ipdavies\AppData\Local\Continuum\anaconda3\envs\tnc_velma\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (9,13,37,43,45) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Get GCM temperature and precipitation projections

In [103]:
# # Get temperature OLD
# forc_dir = config.data_path / 'precip' / 'WRF_frcs_EllsworthCr_forcings'
# forc_file = 'forc_46.40625_-123.90625'
# forc_cols = ['Year', 'Month', 'Day', 'Hour', 'Precip(mm)', 'Temp(C)', 
#              'Wind(m/s)', 'SWrad(W/m2)', 'LWrad(W/m2)', 'pressure(kPa)', 
#              'VaporPress(kPa)']

# cols = ['Temp(C)']
# inds = [forc_cols.index(x) for x in cols]
# arrs = []
# sim_dirs = []
# for sim_dir in os.listdir(forc_dir):
#     if sim_dir == 'pnnl_historical':
#         continue
#     sim_dirs.append(sim_dir)
#     arr = np.loadtxt(forc_dir / sim_dir / forc_file)
#     arrs.append(arr[:, inds])
# stack = np.column_stack(arrs)
# proj_sims_temp = pd.DataFrame(stack, columns=sim_dirs)
# date_arr = pd.DataFrame(arr, columns=forc_cols)
# proj_sims_temp.index = pd.to_datetime(date_arr[['Year', 'Month', 'Day']])

In [112]:
# # Get precipitation OLD
# wrf_dir = config.data_path / 'precip' / 'VIC_WRF_EllsworthCr'
# wrf_file = 'flux_46.40625_-123.90625'
# wrf_cols = ["YEAR","MONTH","DAY","HOUR","OUT_PREC","OUT_PET_SHORT",
#             "OUT_SWE","OUT_EVAP","OUT_RUNOFF","OUT_BASEFLOW",
#             "OUT_SOIL_MOIST0", "OUT_SOIL_MOIST1","OUT_SOIL_MOIST2"]

# cols = ['OUT_PREC']
# inds = [wrf_cols.index(x) for x in cols]
# arrs = []
# sim_dirs = []
# for sim_dir in os.listdir(wrf_dir):
#     if sim_dir == 'pnnl_historical':
#         continue
#     sim_dirs.append(sim_dir)
#     arr = np.loadtxt(wrf_dir / sim_dir / 'sim_avg' / '{}.gz'.format(wrf_file))
#     arrs.append(arr[:, inds])
# stack = np.column_stack(arrs)
# proj_sims_ppt = pd.DataFrame(stack, columns=sim_dirs)
# date_arr = pd.DataFrame(arr, columns=wrf_cols)
# proj_sims_ppt.index = pd.to_datetime(date_arr[['YEAR', 'MONTH', 'DAY']])

In [145]:
# Get precipitation
wrf_dir = config.data_path / 'precip' / 'VIC_WRF_EllsworthCr'
gcm_avg_dir = wrf_dir / 'sim_avg'
cols = ['year', 'month', 'day', 'access1.0_RCP45', 'access1.0_RCP85', 'access1.3_RCP85',
        'bcc-csm1.1_RCP85', 'canesm2_RCP85', 'ccsm4_RCP85', 'csiro-mk3.6.0_RCP85',
        'fgoals-g2_RCP85', 'gfdl-cm3_RCP85', 'giss-e2-h_RCP85', 'miroc5_RCP85',
        'mri-cgcm3_RCP85', 'noresm1-m_RCP85']

arr = np.loadtxt(gcm_avg_dir / 'sim_avg_ppt.gz')
proj_sims_ppt = pd.DataFrame(data=arr, columns=cols)
proj_sims_ppt.index = pd.to_datetime(proj_sims_ppt[['year', 'month', 'day']])
proj_sims_ppt = proj_sims_ppt.drop(['year', 'month', 'day'], axis=1)

In [146]:
# Get temperature
forc_dir = config.data_path / 'precip' / 'WRF_frcs_EllsworthCr_forcings'
gcm_avg_forc_dir = forc_dir / 'sim_avg'
cols = ['year', 'month', 'day', 'access1.0_RCP45', 'access1.0_RCP85', 'access1.3_RCP85',
        'bcc-csm1.1_RCP85', 'canesm2_RCP85', 'ccsm4_RCP85', 'csiro-mk3.6.0_RCP85',
        'fgoals-g2_RCP85', 'gfdl-cm3_RCP85', 'giss-e2-h_RCP85', 'miroc5_RCP85',
        'mri-cgcm3_RCP85', 'noresm1-m_RCP85']

arr = np.loadtxt(gcm_avg_forc_dir / 'sim_avg_temp.gz')
proj_sims_temp = pd.DataFrame(data=arr, columns=cols)
proj_sims_temp.index = pd.to_datetime(proj_sims_temp[['year', 'month', 'day']])
proj_sims_temp = proj_sims_temp.drop(['year', 'month', 'day'], axis=1)

In [637]:
plt.close('all')
colors = sns.color_palette('Dark2', 13)
sim_start = pd.to_datetime('01-01-2025')
sim_end = pd.to_datetime('12-31-2099')
fig = plt.figure(figsize=(9.5, 5))
for i, sim in enumerate(proj_sims_temp):
    sim_data = proj_sims_temp.loc[:, sim]
    sim_subset = sim_data[(sim_data.index >= sim_start) & (sim_data.index <= sim_end)]
    sim_group = sim_subset.groupby(pd.Grouper(freq='10Y')).mean()
    sim_group.plot(label=sim, color=colors[i])
    
ensemble_data = proj_sims_temp.mean(axis=1)
ensemble_subset = ensemble_data[(ensemble_data.index >= sim_start) & (ensemble_data.index <= sim_end)]
ensemble_group = ensemble_subset.groupby(pd.Grouper(freq='10Y')).mean()
ensemble_group.plot(label='Average', color='black', linewidth=2.5)
plt.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=5)
plt.ylabel('Temperature (C)')
fig.suptitle('Fig. 1: GCM air temperature projections (C)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [471]:
plt.close('all')
colors = sns.color_palette('Dark2', 13)
sim_start = pd.to_datetime('01-01-2025')
sim_end = pd.to_datetime('12-31-2099')
fig = plt.figure(figsize=(9.5, 5))
for i, sim in enumerate(proj_sims_temp):
    sim_data = proj_sims_temp.loc[:, sim]
    sim_subset = sim_data[(sim_data.index >= sim_start) & (sim_data.index <= sim_end)]
    sim_group = sim_subset.groupby(pd.Grouper(freq='10Y')).mean()
    ((sim_group * 9/5)+32).plot(label=sim, color=colors[i])
    
ensemble_data = proj_sims_temp.mean(axis=1)
ensemble_subset = ensemble_data[(ensemble_data.index >= sim_start) & (ensemble_data.index <= sim_end)]
ensemble_group = ensemble_subset.groupby(pd.Grouper(freq='10Y')).mean()
((ensemble_group*9/5)+32).plot(label='Average', color='black', linewidth=2.5)
plt.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=5)
plt.ylabel('Temperature (F)')
fig.suptitle('Fig. 1: GCM air temperature projections (F)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [155]:
plt.close('all')
colors = sns.color_palette('Dark2', 13)
sim_start = pd.to_datetime('01-01-2021')
sim_end = pd.to_datetime('12-31-2099')
fig = plt.figure(figsize=(9.5, 5))
for i, sim in enumerate(proj_sims_ppt):
    sim_data = proj_sims_ppt.loc[:, sim]
    sim_subset = sim_data[(sim_data.index >= sim_start) & (sim_data.index <= sim_end)]
    sim_group = sim_subset.groupby(pd.Grouper(freq='Y')).sum()
    sim_group.plot(label=sim, color=colors[i])
    
ensemble_data = proj_sims_ppt.mean(axis=1)
ensemble_subset = ensemble_data[(ensemble_data.index >= sim_start) & (ensemble_data.index <= sim_end)]
ensemble_group = ensemble_subset.groupby(pd.Grouper(freq='Y')).sum()
ensemble_group.plot(label='Average', color='black', linewidth=2.5)
plt.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=5)
plt.ylabel('Precipitation (mm)')
fig.suptitle('Fig. 1: GCM precipitation projections')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [268]:
plt.close('all')
colors = sns.color_palette('Dark2', 13)
sim_start = pd.to_datetime('01-01-2021')
sim_end = pd.to_datetime('12-31-2099')
fig = plt.figure(figsize=(9.5, 5))
for i, sim in enumerate(proj_sims_ppt):
    sim_data = proj_sims_ppt.loc[:, sim]
    sim_subset = sim_data[(sim_data.index >= sim_start) & (sim_data.index <= sim_end)]
    sim_group = sim_subset.groupby(pd.Grouper(freq='m')).sum()
    sim_group.plot(label=sim, color=colors[i])
    
ensemble_data = proj_sims_ppt.mean(axis=1)
ensemble_subset = ensemble_data[(ensemble_data.index >= sim_start) & (ensemble_data.index <= sim_end)]
ensemble_group = ensemble_subset.groupby(pd.Grouper(freq='m')).sum()
ensemble_group.plot(label='Average', color='black', linewidth=2.5)
plt.legend(loc='upper left', bbox_to_anchor=(0, 1.2), fancybox=True, ncol=5)
plt.ylabel('Precipitation (mm)')
fig.suptitle('Fig. 1: GCM precipitation projections')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [264]:
fig = plt.figure(figsize=(9.5, 5))
ensemble_group = ensemble_subset.groupby(pd.Grouper(freq='m')).sum()
ensemble_group.plot(label='Average', linewidth=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

## Scaling historical temperature normals to GCM projections

### Scaling temperature with simple regression

In [212]:
proj_start = pd.to_datetime('01-01-2021')
proj_end = pd.to_datetime('12-31-2099')

# Get PRISM daily normals
piv = pd.pivot_table(temp, columns=temp.index.year, index=temp.index.dayofyear, values='temp')
normals_temp_d = piv.mean(axis=1)

# Repeat historical daily normals for every year in projected range - these values will be scaled
# Merging on day of year will take leap year into account by not merging Feb 29 on non-leap years
proj_rng = pd.DataFrame(index=pd.date_range(proj_start, proj_end))
proj_rng['doy'] = proj_rng.index.dayofyear
proj_temp = proj_rng.merge(pd.DataFrame(normals_temp_d, columns=['normals_temp_d']), 
               left_on='doy', right_index=True, how='left').drop('doy', axis=1)

# Get average GCM daily temps
proj_sims_temp_edit = proj_sims_temp[(proj_sims_temp.index >= proj_start) & (proj_sims_temp.index <= proj_end)]
gcm_temp_d_mean = pd.DataFrame(proj_sims_temp_edit.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_temp_d_mean'])

# Convert day of year to signal
day = 24 * 60 * 60
year = 365.2425 * day
timestamp_secs = pd.to_datetime(gcm_temp_d_mean.index)
timestamp_secs = timestamp_secs.map(datetime.datetime.timestamp)
gcm_temp_d_mean['doy_cos'] = np.cos(timestamp_secs * (2 * np.pi / year))
gcm_temp_d_mean['doy_sin'] = np.sin(timestamp_secs * (2 * np.pi / year))
gcm_temp_d_mean['year'] = gcm_temp_d_mean.index.year

In [213]:
# Train regression model
X = gcm_temp_d_mean.copy()
y = X.pop('gcm_temp_d_mean')
X = sm.add_constant(X)
X['normals_temp_d'] = proj_temp['normals_temp_d']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = sm.add_constant(X_train)
olsmodel = sm.OLS(y_train, X_train).fit()
print(olsmodel.summary())

                            OLS Regression Results                            
Dep. Variable:        gcm_temp_d_mean   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                 8.528e+04
Date:                Fri, 26 Feb 2021   Prob (F-statistic):               0.00
Time:                        10:30:54   Log-Likelihood:                -27391.
No. Observations:               19332   AIC:                         5.479e+04
Df Residuals:                   19327   BIC:                         5.483e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               -76.6957      0.65

In [214]:
y_pred = olsmodel.predict(X)

In [215]:
fig = plt.figure(figsize=(8, 3))
((y.groupby(pd.Grouper(freq='m')).mean()*9/5)+32).plot(label='gcm')
((X['normals_temp_d'].groupby(pd.Grouper(freq='m')).mean() * 9/5)+32).plot(label='normals')
((y_pred.groupby(pd.Grouper(freq='m')).mean()*9/5)+32).plot(label='proj')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

This method basically reproduces the projection and doesn't use the historical normals as predictors

### Scaling using a constant

Based on the [CIG Puget Sound projections report](https://cig.uw.edu/wp-content/uploads/sites/2/2014/11/ps-sok_sec02_climate_2015.pdf), going to scale seasons by a different constant. The constants are projected increases relative to 1970-1999 normals though, so we will need to use that data as the baseline values.

In [416]:
# Examining GCM normals 1990-2020 compared to PRISM normals 1990-2020 to see if they are comparable, before using GCM 1970-1999
proj_sims_temp_hist = proj_sims_temp[(proj_sims_temp.index >= temp.index.min()) & (proj_sims_temp.index <= temp.index.max())]
gcm_hist_d_mean = pd.DataFrame(proj_sims_temp_hist.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_hist_d_mean'])

piv = pd.pivot_table(gcm_hist_d_mean, columns=gcm_hist_d_mean.index.year, index=gcm_hist_d_mean.index.dayofyear, values='gcm_hist_d_mean')
gcm_normals = piv.mean(axis=1)

proj_sims_temp_hist = proj_sims_temp[(proj_sims_temp.index >= pd.to_datetime('01-01-1970')) & 
                                     (proj_sims_temp.index <= pd.to_datetime('12-31-1999'))]
gcm_hist_d_mean = pd.DataFrame(proj_sims_temp_hist.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_hist_d_mean'])
piv = pd.pivot_table(gcm_hist_d_mean, columns=gcm_hist_d_mean.index.year, index=gcm_hist_d_mean.index.dayofyear, values='gcm_hist_d_mean')
gcm_normals_1970 = piv.mean(axis=1)

piv = pd.pivot_table(temp, columns=temp.index.year, index=temp.index.dayofyear, values='temp')
normals_temp_d = piv.mean(axis=1)

normals = pd.concat([normals_temp_d, gcm_normals, gcm_normals_1970], axis=1)
normals.columns=['prism_normals', 'gcm_normals', 'gcm_normals_1970']
normals.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [418]:
proj_start = pd.to_datetime('01-01-2021')
proj_end = pd.to_datetime('12-31-2099')

# Get GCM daily normals 1970-1999
proj_sims_temp_hist = proj_sims_temp[(proj_sims_temp.index >= pd.to_datetime('01-01-1970')) & 
                                     (proj_sims_temp.index <= pd.to_datetime('12-31-1999'))]
gcm_hist_d_mean = pd.DataFrame(proj_sims_temp_hist.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_hist_d_mean'])
piv = pd.pivot_table(gcm_hist_d_mean, columns=gcm_hist_d_mean.index.year, index=gcm_hist_d_mean.index.dayofyear, values='gcm_hist_d_mean')
gcm_normals_d_1970 = piv.mean(axis=1)

# Repeat historical daily normals for every year in projected range - these values will be scaled
# Merging on day of year will take leap year into account by not merging Feb 29 on non-leap years
proj_rng = pd.DataFrame(index=pd.date_range(proj_start, proj_end))
proj_rng['doy'] = proj_rng.index.dayofyear
proj_temp = proj_rng.merge(pd.DataFrame(gcm_normals_d_1970, columns=['gcm_normals_d_1970']), 
               left_on='doy', right_index=True, how='left').drop('doy', axis=1)

# Get average GCM daily temps
proj_sims_temp_edit = proj_sims_temp[(proj_sims_temp.index >= proj_start) & (proj_sims_temp.index <= proj_end)]
gcm_temp_d_mean = pd.DataFrame(proj_sims_temp_edit.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_temp_d_mean'])

In [439]:
# # Calculate constants
proj_years = len(np.unique(pd.date_range(proj_start, proj_end).year))
fall_yearly_constant = 7.1 / proj_years
winter_yearly_constant = 6.65 / proj_years
spring_yearly_constant = 6.6 / proj_years
summer_yearly_constant = 8.75 / proj_years

# proj_years = len(np.unique(pd.date_range(proj_start, proj_end).year))
# fall_yearly_constant = 5 / proj_years
# winter_yearly_constant = 5 / proj_years
# spring_yearly_constant = 5 / proj_years
# summer_yearly_constant = 5 / proj_years

In [440]:
# Apply constants
constants = []
for date in proj_temp.index:
    if date.month in [12, 1, 2]:
        constants.append(winter_yearly_constant * (date.year - 2020))
    if date.month in [3, 4, 5]:
        constants.append(spring_yearly_constant * (date.year - 2020))
    if date.month in [6, 7, 8]:
        constants.append(summer_yearly_constant * (date.year - 2020))
    if date.month in [9, 10, 11]:
        constants.append(fall_yearly_constant * (date.year - 2020))

proj_temp['constants'] = constants
proj_temp['proj_temp'] = proj_temp['gcm_normals_d_1970'] + proj_temp['constants']

In [441]:
plt.close('all')
fig = plt.figure(figsize=(8, 3))
gcm_temp_d_mean['gcm_temp_d_mean'].groupby(pd.Grouper(freq='m')).mean().plot(label='gcm average')
proj_temp['proj_temp'].groupby(pd.Grouper(freq='m')).mean().plot(label='proj w/ seasonal constants')
proj_temp['gcm_normals_d_1970'].groupby(pd.Grouper(freq='m')).mean().plot(label='normals')

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [442]:
fig = plt.figure(figsize=(8, 3))
((gcm_temp_d_mean['gcm_temp_d_mean'].groupby(pd.Grouper(freq='m')).mean()* 9/5)+32).plot(label='gcm average')
((proj_temp['proj_temp'].groupby(pd.Grouper(freq='m')).mean()* 9/5)+32).plot(label='proj w/ seasonal constants')
((proj_temp['gcm_normals_d_1970'].groupby(pd.Grouper(freq='m')).mean()* 9/5)+32).plot(label='normals')

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [443]:
fig = plt.figure(figsize=(8, 3))
((gcm_temp_d_mean['gcm_temp_d_mean'].groupby(pd.Grouper(freq='y')).mean()* 9/5)+32).plot(label='gcm average')
((proj_temp['proj_temp'].groupby(pd.Grouper(freq='y')).mean()* 9/5)+32).plot(label='proj w/ seasonal constants')
((proj_temp['gcm_normals_d_1970'].groupby(pd.Grouper(freq='y')).mean()* 9/5)+32).plot(label='normals')

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Why does scaling with seasonal constants lead to such a higher projected temperature?

## Scaling historical precipitation normals to GCM projections

### Scaling using monthly means

In [223]:
proj_start = pd.to_datetime('01-01-2021')
proj_end = pd.to_datetime('12-31-2099')

# Get PRISM daily normals
piv_d = pd.pivot_table(prism_precip_mean, columns=prism_precip_mean.index.year, index=prism_precip_mean.index.dayofyear, values='avg_precip')
normals_precip_d = piv_d.mean(axis=1)

# Repeat daily normals for every year in projected range - these values will be scaled
# Merging on day of year will take leap year into account by not merging Feb 29 on non-leap years
proj_rng = pd.DataFrame(index=pd.date_range(proj_start, proj_end))
proj_rng['doy'] = proj_rng.index.dayofyear
proj_precip = proj_rng.merge(pd.DataFrame(normals_precip_d, columns=['normals_precip_d']), 
               left_on='doy', right_index=True, how='left').drop('doy', axis=1)

# Get PRISM monthly normals
proj_years = len(pd.date_range(proj_start, proj_end, freq='y'))
piv_m = pd.pivot_table(prism_precip_mean, columns=prism_precip_mean.index.year, index=prism_precip_mean.index.month, values='avg_precip', aggfunc=np.sum)
# normals_m = pd.DataFrame(piv_m.mean(axis=1), columns=['normals_m'])
normals_m = np.tile(piv_m.mean(axis=1), proj_years)

# # Join daily and monthly normals
# proj_precip.index = proj_precip.index.strftime('%m')
# proj_precip.join(normals_m, how='left')

# Get GCM monthly means
gcm_precip_m = proj_sims_ppt.groupby(pd.Grouper(freq='M')).sum()
gcm_precip_m = gcm_precip_m[(gcm_precip_m.index >= proj_start) & (gcm_precip_m.index <= proj_end)]
gcm_precip_m_mean = gcm_precip_m.mean(axis=1)

# For each projected month, find the % increase from historical PRISM to projected GCM
scale_m = (gcm_precip_m_mean- normals_m) / normals_m
scale_m_precip = pd.DataFrame(scale_m, columns=['scale_m_precip'])

# Apply scaling factor to historical daily normals to get projected values
proj_precip.index = proj_precip.index.strftime('%Y-%m')
scale_m_precip.index = scale_m_precip.index.strftime('%Y-%m')
proj_precip_scaled = proj_precip.merge(scale_m_precip, left_index=True, right_index=True, how='left')
proj_precip_scaled['proj_precip_scaled_m'] = proj_precip_scaled['normals_precip_d'] + (proj_precip_scaled['normals_precip_d'] * proj_precip_scaled['scale_m_precip'])
proj_precip_scaled.index = proj_rng.index

In [224]:
normals_m_df = pd.DataFrame(normals_m, index=gcm_precip_m_mean.index)
fig = plt.figure(figsize=(8, 3))
ax = proj_precip_scaled.groupby(pd.Grouper(freq='Y')).sum()['proj_precip_scaled_m'].plot()
normals_m_df.groupby(pd.Grouper(freq='Y')).sum().plot(ax=ax)
fig.suptitle('Projected precipitation scaled using monthly means')
# Perhaps using monthly means instead of yearly means allows for too much variation

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Projected precipitation scaled using monthly means')

### Scaling using yearly means

In [581]:
proj_start = pd.to_datetime('01-01-2020')
proj_end = pd.to_datetime('12-31-2099')

# Get GCM yearly mean
gcm_ppt_y_mean = proj_sims_ppt.groupby(pd.Grouper(freq='Y')).sum().mean(axis=1)
gcm_ppt_y_mean = gcm_ppt_y_mean[(gcm_ppt_y_mean.index >= proj_start) & (gcm_ppt_y_mean.index <= proj_end)]

# Get PRISM daily mean
piv = pd.pivot_table(prism_precip_mean, columns=prism_precip_mean.index.year, index=prism_precip_mean.index.dayofyear, values='avg_precip')
prism_precip_d_mean = piv.mean(axis=1)

# Get PRISM yearly mean
prism_precip_y_mean = prism_precip_mean.groupby(pd.Grouper(freq='Y')).sum()['avg_precip']

In [226]:
# Get yearly mean of historical daily normals
hist_precip_y_mean = proj_precip_scaled['normals_precip_d'].groupby(pd.Grouper(freq='Y')).mean()

# For each projected month, find the % increase from historical PRISM to projected GCM
scale_y = (gcm_ppt_y_mean- hist_precip_y_mean) / hist_precip_y_mean
scale_y_precip = pd.DataFrame(scale_y, columns=['scale_y_precip'])

# Apply scaling factor to historical daily normals to get projected values
proj_precip_scaled.index = proj_precip_scaled.index.strftime('%Y')
scale_y_precip.index = scale_y_precip.index.strftime('%Y')
proj_precip_scaled = proj_precip_scaled.merge(scale_y_precip, left_index=True, right_index=True, how='left')
proj_precip_scaled
proj_precip_scaled['proj_precip_scaled_y'] = proj_precip_scaled['normals_precip_d'] + (proj_precip_scaled['normals_precip_d'] * proj_precip_scaled['scale_y_precip'])
proj_precip_scaled.index = proj_rng.index

In [450]:
x = proj_precip_scaled.groupby(pd.Grouper(freq='Y')).sum()
x['year'] = x.index.year
m, b = np.polyfit(x['year'], x['proj_precip_scaled_m'], 1)
z = m*x['year']+b

fig = plt.figure(figsize=(8, 3))
proj_precip_scaled['proj_precip_scaled_y'].groupby(pd.Grouper(freq='Y')).mean().plot(label='Proj yearly mean (scaled with yearly)')
proj_precip_scaled.groupby(pd.Grouper(freq='Y')).sum()['proj_precip_scaled_m'].plot(label='Proj yearly mean (scaled with monthly)')
z.plot(label='Projected trend')
proj_precip_scaled['normals_precip_d'].groupby(pd.Grouper(freq='Y')).sum().plot(label='Normals yearly mean')
fig.suptitle('Projected precipitation scaled with monthly and yearly means')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

These are still very naive scaled projections. Want some way to capture (1) seasonal variation of temperature, which is projected to rise higher in summer than in winter, and (2) seasonal variation of precipitation which is projected to lower for summer and rise for winter.


## Scaling using bias-correction for temperature

http://www.ccafs-climate.org/bias_correction/#Bias_correction

$$ T_{BC}(t) = \overline{O_{ref}} + \frac{\sigma_{O,ref}}{\sigma_{T,ref}}(T_{raw}(t) - \overline{T_{ref}}) $$ 

Might want to do the individual GCMs

Need to have reference and future periods of equal length. So will do chunks to match 30 year normals: 2010-2039, 2040-2069, 2070-2099


In [671]:
starts = [pd.to_datetime(x) for x in ['01-01-2010', '01-01-2040', '01-01-2070']]
ends = [pd.to_datetime(x) for x in ['12-31-2039', '12-31-2069', '12-31-2099']]

normals_start = pd.to_datetime('1981-01-01')
normals_end = pd.to_datetime('2010-12-31')

# Get PRISM daily normals
z = temp[(temp.index >= normals_start) & (temp.index <= normals_end)]
piv_d = pd.pivot_table(z, columns=z.index.year, 
                       index=z.index.dayofyear, values='temp')
normals_temp_d = piv_d.mean(axis=1)

# Get historical daily normals from GCM
proj_sims_temp_hist = proj_sims_temp[(proj_sims_temp.index >= z.index.min()) & 
                                     (proj_sims_temp.index <= z.index.max())]
gcm_hist_d_mean = pd.DataFrame(proj_sims_temp_hist.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_hist_d_mean'])
piv = pd.pivot_table(gcm_hist_d_mean, columns=gcm_hist_d_mean.index.year, index=gcm_hist_d_mean.index.dayofyear, values='gcm_hist_d_mean')
gcm_normals_d = piv.mean(axis=1)

proj_sims_temp_d = proj_sims_temp.groupby(pd.Grouper(freq='d')).mean()
o_ref_mean = np.mean(normals_temp_d)
sigma_o_ref = np.std(normals_temp_d)
t_ref_mean = np.mean(gcm_normals_d)
sigma_t_ref = np.std(gcm_normals_d)

t_raw_list = []
t_bc_list = [] 
for start, end in zip(starts, ends):
    t_raw_sims = proj_sims_temp_d[(proj_sims_temp_d.index >= start) & 
                            (proj_sims_temp_d.index <= end)]
    t_raw_chunk = t_raw_sims.mean(axis=1)
    
    t_bc_chunk = o_ref_mean + (sigma_o_ref / sigma_t_ref)*(t_raw_chunk - t_ref_mean)
    
    t_raw_list.append(t_raw_chunk)
    t_bc_list.append(t_bc_chunk)

rng = pd.date_range(starts[0], ends[2])
t_bc = pd.DataFrame(data=np.concatenate(t_bc_list, axis=0), columns=['t_bc'], index=rng)
t_raw = pd.DataFrame(data=np.concatenate(t_raw_list, axis=0), columns=['t_raw'], index=rng)

In [729]:
display('Means of raw projected chunks: ', [x.mean() for x in t_raw_list])
display('Means of historical temp chunks: ', [x.mean() for x in t_bc_list])

'Means of raw projected chunks: '

[10.360872703435106, 11.400637302216856, 12.708300550754348]

'Means of historical temp chunks: '

[10.646711049501109, 11.741296179601669, 13.117904643387789]

In [727]:
plt.close('all')
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
axes[0].title.set_text('Yearly average')
axes[0].plot(t_bc.groupby(pd.Grouper(freq='y')).mean(), label="Bias-correction scaled")
axes[0].plot(t_raw.groupby(pd.Grouper(freq='y')).mean(), label="Ensemble average")
axes[1].title.set_text('Daily average')
axes[1].plot(t_bc)
axes[1].plot(t_raw)
axes[0].legend()
axes[0].get_xaxis().set_visible(False)
plt.ylabel('Degrees (C)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Scaling using change factor

In [691]:
starts = [pd.to_datetime(x) for x in ['01-01-2010', '01-01-2040', '01-01-2070']]
ends = [pd.to_datetime(x) for x in ['12-31-2039', '12-31-2069', '12-31-2099']]

normals_start = pd.to_datetime('1981-01-01')
normals_end = pd.to_datetime('2010-12-31')

# Get PRISM daily normals
normals_start = pd.to_datetime('1981-01-01')
normals_end = pd.to_datetime('2010-12-31')

# Get PRISM daily normals
z = temp[(temp.index >= normals_start) & (temp.index <= normals_end)]
piv_d = pd.pivot_table(z, columns=z.index.year, 
                       index=z.index.dayofyear, values='temp')
normals_temp_d = piv_d.mean(axis=1)

# Get historical daily normals from GCM
proj_sims_temp_hist = proj_sims_temp[(proj_sims_temp.index >= prism_precip_mean.index.min()) & 
                                     (proj_sims_temp.index <= prism_precip_mean.index.max())]
gcm_hist_d_mean = pd.DataFrame(proj_sims_temp_hist.groupby(pd.Grouper(freq='d')).mean().mean(axis=1), columns=['gcm_hist_d_mean'])
piv = pd.pivot_table(gcm_hist_d_mean, columns=gcm_hist_d_mean.index.year, index=gcm_hist_d_mean.index.dayofyear, values='gcm_hist_d_mean')
gcm_normals_d = piv.mean(axis=1)

proj_sims_temp_d = proj_sims_temp.groupby(pd.Grouper(freq='d')).mean()
o_ref = z['temp']
sigma_o_ref = np.std(normals_temp_d)
t_ref_mean = np.mean(gcm_normals_d)
sigma_t_ref = np.std(gcm_normals_d)

t_raw_list = []
t_cf_list = [] 
for start, end in zip(starts, ends):
    t_raw_sims = proj_sims_temp_d[(proj_sims_temp_d.index >= start) & 
                            (proj_sims_temp_d.index <= end)]
    t_raw_chunk = t_raw_sims.mean(axis=1)
    t_raw_mean = np.mean(t_raw_chunk)
    sigma_t_raw = np.std(t_raw_chunk)
    t_cf_chunk = t_raw_mean + (sigma_t_raw / sigma_t_ref)*(o_ref - t_ref_mean)
    
    t_raw_list.append(t_raw_chunk)
    t_cf_list.append(t_cf_chunk.values)

# Need to tack on an extra day to account for misaligned leap years between o_ref and t_raw
t_cf_list[0] = np.append(t_cf_list[0], t_cf_list[0][-1])
rng = pd.date_range(starts[0], ends[2])
t_cf = pd.DataFrame(data=np.concatenate(t_cf_list, axis=0), columns=['t_cf'], index=rng)
t_raw = pd.DataFrame(data=np.concatenate(t_raw_list, axis=0), columns=['t_raw'], index=rng)

In [728]:
display('Means of raw projected chunks: ', [x.mean() for x in t_raw_list])
display('Means of historical temp chunks: ', [x.mean() for x in t_cf_list])

'Means of raw projected chunks: '

[10.360872703435106, 11.400637302216856, 12.708300550754348]

'Means of historical temp chunks: '

[10.443930393547777, 11.488464089681786, 12.796621482233402]

In [726]:
plt.close('all')
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
axes[0].title.set_text('Yearly average')
axes[0].plot(t_cf.groupby(pd.Grouper(freq='y')).mean(), label="Change factor scaled")
axes[0].plot(t_raw.groupby(pd.Grouper(freq='y')).mean(), label="Ensemble average")
axes[1].title.set_text('Daily average')
axes[1].plot(t_cf)
axes[1].plot(t_raw)
axes[0].legend()
axes[0].get_xaxis().set_visible(False)
plt.ylabel('Degrees (C)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The change factor method looks superior to bias correction because it contains more variation - natural heat and cold waves that were present in the 1981-2010 data, but scaled up so that the means are roughly equal to the projected data.

## Exports

In [470]:
velma_start = pd.to_datetime('01-01-2005')
velma_end = pd.to_datetime('12-31-2099')

# Export ensemble average temperature, 2022 - 2099
ensemble_temp = proj_sims_temp[(proj_sims_temp.index >= velma_start) & (proj_sims_temp.index <= velma_end)]
ensemble_temp_mean = ensemble_temp.mean(axis=1)
outfile = config.velma_data / 'temp' / 'gcm_avg_temp_05_99.csv'
ensemble_temp_mean.to_csv(outfile, header=False, index=False)

# Export ensemble average precipitation, 2022 - 2099
ensemble_precip = proj_sims_ppt[(proj_sims_ppt.index >= velma_start) & (proj_sims_ppt.index <= velma_end)]
ensemble_precip_mean = ensemble_precip.mean(axis=1)
outfile = config.velma_data / 'precip' / 'gcm_avg_precip_05_99.csv'
ensemble_precip_mean.to_csv(outfile, header=False, index=False)

# Export 1990-2020 normals, repeated to 2099
# Get PRISM daily normals
piv_d = pd.pivot_table(prism_precip_mean, columns=prism_precip_mean.index.year, index=prism_precip_mean.index.dayofyear, values='avg_precip')
normals_precip_d = piv_d.mean(axis=1)
# Repeat daily normals for every year in projected range - these values will be scaled
# Merging on day of year will take leap year into account by not merging Feb 29 on non-leap years
proj_rng = pd.DataFrame(index=pd.date_range(proj_start, proj_end))
proj_rng['doy'] = proj_rng.index.dayofyear
proj_precip = proj_rng.merge(pd.DataFrame(normals_precip_d, columns=['normals_precip_d']), 
               left_on='doy', right_index=True, how='left').drop('doy', axis=1)
outfile = config.velma_data / 'precip' / 'prism_gauge_avg_1990_20_normals_05_99.csv'
proj_precip.to_csv(outfile, header=False, index=False)

In [601]:
# Repeating 2000-2019 precipitation to 2099
hist_start = pd.to_datetime('01-01-2000')
hist_end = pd.to_datetime('12-31-2019')
proj_years = len(np.unique(pd.date_range(proj_start, proj_end).year))

z = prism_precip_mean[(prism_precip_mean.index >= hist_start) & (prism_precip_mean.index <= hist_end)]['avg_precip']
x = np.tile(z, int(proj_years / len(np.unique(z.index.year))))
precip_tiled = pd.DataFrame(x, columns=['avg_precip'], index=pd.date_range(proj_start, proj_end))

outfile = config.velma_data / 'precip' / 'prism_gauge_avg_20_99.csv'
precip_tiled.to_csv(outfile, header=False, index=False)

In [707]:
# Export change factor scaled temperature
t_cf_proj = t_cf[(t_cf.index >= proj_start) & (t_cf.index <= proj_end)]
outfile = config.velma_data / 'temp' / 'prism_temp_cf_20_99.csv'
t_cf.to_csv(outfile, header=False, index=False)